In [ ]:
import os
import uuid
import flask
import urllib
from PIL import Image
from tensorflow.keras.models import load_model
from flask import Flask , render_template  , request , send_file
from tensorflow.keras.preprocessing.image import load_img , img_to_array


app = Flask(__name__)

BASE_DIR = os.path.dirname('static')
model = load_model('Spotflock_2.h5')
ALLOWED_EXT = set(['jpg' , 'jpeg' , 'png' , 'jfif'])

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1] in ALLOWED_EXT

classes = ['Gateway of India', 'Golden Temple', 'Hawa Mahal', 'Humayun Tomb', 'India Gate', 'Qutub Minar', 'Red Fort', 'Sanchi Stupa', 'Statue of Unity', 'Taj Mahal']
def predict(filename , model):
    img = load_img(filename , target_size = (224 , 224))
    img = img_to_array(img)
    img = img.reshape(1 , 224 ,224 ,3)
    img = img.astype('float32')
    img = img/255.0
    result = model.predict(img)
    dict_result = {}
    for i in range(10):
        dict_result[result[0][i]] = classes[i]
    res = result[0]
    res.sort()
    res = res[::-1]
    prob = res[:3]
    
    prob_result = []
    class_result = []
    for i in range(3):
        prob_result.append((prob[i]*100).round(2))
        class_result.append(dict_result[prob[i]])
    return class_result , prob_result
@app.route('/')
def home():
        return render_template("index.html")
@app.route('/success' , methods = ['GET' , 'POST'])
def success():
    error = ''
    target_img = os.path.join(os.getcwd() , 'static/images')
    if request.method == 'POST':
        if(request.form):
            link = request.form.get('link')
            try :
                resource = urllib.request.urlopen(link)
                unique_filename = str(uuid.uuid4())
                filename = unique_filename+".jpg"
                img_path = os.path.join(target_img , filename)
                output = open(img_path , "wb")
                output.write(resource.read())
                output.close()
                img = filename
                class_result , prob_result = predict(img_path , model)
                predictions = {
                      "class1":class_result[0],
                        "class2":class_result[1],
                        "class3":class_result[2],
                        "prob1": prob_result[0],
                        "prob2": prob_result[1],
                        "prob3": prob_result[2],
                }
            except Exception as e : 
                print(str(e))
                error = 'This image from this site is not accesible or inappropriate input'
            if(len(error) == 0):
                return  render_template('success.html' , img  = img , predictions = predictions)
            else:
                return render_template('index.html' , error = error) 
            
        elif (request.files):
            file = request.files['file']
            if file and allowed_file(file.filename):
                file.save(os.path.join(target_img , file.filename))
                img_path = os.path.join(target_img , file.filename)
                img = file.filename
                class_result , prob_result = predict(img_path , model)
                predictions = {
                      "class1":class_result[0],
                        "class2":class_result[1],
                        "class3":class_result[2],
                        "prob1": prob_result[0],
                        "prob2": prob_result[1],
                        "prob3": prob_result[2],
                }
            else:
                error = "Please upload images of jpg , jpeg and png extension only"
            if(len(error) == 0):
                return  render_template('success.html' , img  = img , predictions = predictions)
            else:
                return render_template('index.html' , error = error)
    else:
        return render_template('index.html')
if __name__ == "__main__":
    app.run(port=1504,debug = True, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:1504/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Oct/2022 10:44:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2022 10:44:39] "GET /static/css/normalize.css HTTP/1.1" 404 -
127.0.0.1 - - [11/Oct/2022 10:44:39] "GET /static/css/grid.css HTTP/1.1" 404 -
127.0.0.1 - - [11/Oct/2022 10:44:39] "GET /static/css/style.css HTTP/1.1" 404 -
127.0.0.1 - - [11/Oct/2022 10:44:39] "GET /static/css/front.jpg HTTP/1.1" 404 -
127.0.0.1 - - [11/Oct/2022 10:44:39] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [11/Oct/2022 10:44:55] "POST /success HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2022 10:44:55] "GET /static/css/normalize.css HTTP/1.1" 404 -
127.0.0.1 - - [11/Oct/2022 10:44:55] "GET /static/css/grid.css HTTP/1.1" 404 -
127.0.0.1 - - [11/Oct/2022 10:44:55] "GET /static/css/styleSucc.css HTTP/1.1" 404 -
127.0.0.1 - - [11/Oct/2022 10:44:55] "GET /static/images/images.jpeg HTTP/1.1" 200 -


In [1]:
pip freeze > requirements2.txt

Note: you may need to restart the kernel to use updated packages.
